In [1]:

                                                
#   @author     : EL OMARI MOHAMMED HACHIM - NACHID AYMEN 
# Group : INFO 2 
                                 




import tkinter as tk
from tkinter import *
import json
import random as rd
import time


class SpaceInvaders(object):
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.root.resizable(0, 0)
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top", fill="both")
        self.game = Game(self.frame)
            
    def play(self):
        
        def lancer(event):
            self.input_name.config(state='disabled') 
            self.game.score.set_name(self.input_name.get()) 
            self.root.bind("<Key>", self.game.stop_def) 
            self.game.start_animation() 
                
        Lname = Label(self.frame, text="Enter Votre Pseudo : ").pack(side = TOP)  
        self.input_name = Entry(self.frame,bd =5)
        self.input_name.pack()
        self.input_name.focus_set()
        self.input_name.bind('<Return>',lancer) 
        self.root.mainloop()
                        
#-------------------Game-------------------------
class Game(object):
    def __init__(self, frame):
        self.frame = frame
        self.fleet = Fleet()
        self.defender = Defender()
        self.score = Score()
        
        # Charger l'image earth.gif
        self.bg_image = PhotoImage(file="earth.gif")  # Charger l'image d'arrière-plan
        
        # Adapter la taille du canevas à l'image (600x600)
        self.canvas_height = 600
        self.canvas_width = 600
        self.canvas = tk.Canvas(self.frame, width=self.canvas_width, height=self.canvas_height, bg='grey')
        
        # Placer l'image d'arrière-plan dans le canevas
        self.canvas.create_image(0, 0, anchor=NW, image=self.bg_image)
        
        self.canvas.pack(padx=5, pady=5, side="bottom")
    
        self.defender.install_in(self.canvas)
        self.defender.display_defender_lifes(self.canvas)
        self.fleet.install_in(self.canvas)
    
    
    def stop_def(self, event):
        xcond = self.defender.xi  
        if event.keysym == 'Left':
            if xcond - self.defender.move_delta > 0:    
                self.defender.xi = self.defender.xi - self.defender.move_delta    
                self.defender.move_in(self.canvas,-self.defender.move_delta)                
        elif event.keysym == 'Right':
            if xcond + self.defender.move_delta < self.defender.canvas_width:   
                self.defender.xi = self.defender.xi + self.defender.move_delta    
                self.defender.move_in(self.canvas,self.defender.move_delta)
        elif event.keysym == 'space':    
            if len(self.fleet.aliens_fleet) != 0 and self.defender.lifes > 0: 
                self.defender.fire(self.canvas)
                

    def Bienvenue(self):
        text1 = "Bonjour : ' "+str(self.score.name) +" ' "        
        label = Label(self.canvas, text=text1,bg="grey", fg="white", font="Arial 13")

        self.canvas.create_window(110,25, window=label)

    
    
    def affiche_score(self):
        self.score_actuel = StringVar()  
        score_label = Label(self.canvas, textvariable = self.score_actuel, bg="grey", fg="white", font="Arial 13")
        self.score_actuel.set("score : " + str(self.score.get_points())) 
        self.canvas.create_window(1050, 25, window = score_label)
    
   
    def saveFile(self):
        self.score.delai = int(time.time()) - self.score.start_time 
        self.score.toFile("LastScore.json") 
        lastScore = self.score.fromFile("LastScore.json")
        self.resultats = Resultats.fromFile("AllResults.json")
        self.resultats.ajout(lastScore) 
        self.resultats.toFile("AllResults.json") 
    
    
    def game_over(self):
        self.score.winning = False 
        self.saveFile() 
        w,h = int(self.canvas.cget("width"))//2 , int(self.canvas.cget("height"))//2
        self.canvas.delete("all") 
        self.canvas.create_text(w,h,text="Game Over", fill="white", font="Times 20 bold")
    
    
    def victoire(self):
        self.score.winning = True 
        self.saveFile()
        w,h = int(self.canvas.cget("width"))//2 , int(self.canvas.cget("height"))//2
        self.canvas.delete("all")
        self.canvas.create_text(w,h,text="Vous avez gagné !", fill="white", font="Times 20 bold")

        

    
    def start_animation(self):
        self.Bienvenue() 
        self.affiche_score() 
        self.fleet.previous_shoot_time = float(time.time()) 
        self.animation() 
    
    def animation(self):
        self.score_actuel.set("score : "+str(self.score.get_points()))
        if len(self.fleet.aliens_fleet)!= 0: 
            x1,y1,x2,y2 = self.canvas.bbox("alien")
            
            if y2<=int(self.canvas.cget("height"))-self.defender.height and self.defender.lifes > 0:
                self.defender.move_bullet(self.canvas) 
                self.defender.manage_defender_touched_by(self.canvas,self.fleet)
                self.fleet.move_aliens_bullets(self.canvas)
                self.fleet.move_in(self.canvas)
                self.fleet.manage_touched_aliens_by(self.canvas,self.defender,self.score)
                self.fleet.install_aliens_bullets(self.canvas)
                self.fleet.animation_aliens_img(self.canvas)
                
                self.canvas.after(50,self.animation) 
            else: 
                self.game_over()
        else: 
            self.victoire()
            
            
#-------------------Fleet-------------------------
class Fleet(object):
    def __init__(self):
        self.width = 1220 
        self.height = 600 
        
        self.aliens_lines = 5
        self.aliens_columns = 6
        self.aliens_inner_gap = 10
        fleet_size = self.aliens_lines * self.aliens_columns
        self.aliens_fleet = [None] * fleet_size
       
        self.alien_x_delta = 5
        self.alien_y_delta = 15
       
        self.animation_start_time = float(time.time())
      
        self.temps_ecart = 1     
        self.previous_shoot_time = None    
        self.max_fired_bullets = 5  
        self.aliens_fired_bullets = []
    
    
    def get_width(self):
        return self.width
    def get_height(self):
        return self.height
    
  
    def install_in(self, canvas):
        x, y = 50 , 75   
        pos = 0 
        for i in range(0,self.aliens_lines):
            for j in range(0,self.aliens_columns):
                alien = Alien()
                self.aliens_fleet[pos] = alien.install_in(canvas,x,y) 
                pos += 1
                x += self.aliens_inner_gap + alien.alien_width
            x = 50
            y += self.aliens_inner_gap + alien.alien_height
            
    
    def move_in(self, canvas): 
        if len(self.aliens_fleet) != 0:
            x1,y1,x2,y2 = canvas.bbox("alien")
            if x2 >= int(canvas.cget("width")): 
                self.alien_x_delta = -self.alien_x_delta
                dy = self.alien_y_delta
            elif x1 <= 0:
                self.alien_x_delta = -self.alien_x_delta
                dy = self.alien_y_delta
            else:
                dy = 0
            for i in range(0,len(self.aliens_fleet)):
                self.aliens_fleet[i].move_in(canvas,self.alien_x_delta,dy)
    
    
    def manage_touched_aliens_by(self,canvas,defender,score):
        sortir1 = False 
        sortir2 = False 
        for i in range(len(self.aliens_fleet)): 
            x1,y1,x2,y2 = canvas.bbox(self.aliens_fleet[i].id) 
            overlapped = canvas.find_overlapping(x1, y1, x2, y2)
            if len(overlapped) > 1 : 
                for j in range(len(defender.fired_bullets)):
                    for k in range(len(overlapped)):
                        if defender.fired_bullets[j].id == overlapped[k]: 
                            self.effet_boom(canvas,defender.fired_bullets[j]) 
                            self.aliens_fleet[i].touched_by(canvas,defender.fired_bullets[j]) 
                            score.refresh_score(time.time()) 
                            del defender.fired_bullets[j]   
                            del self.aliens_fleet[i]
                            sortir1 = True
                            sortir2 = True
                            break; 
                    if sortir2 == True: 
                        break;
                if sortir1 == True: 
                    break;
    
    
    def install_aliens_bullets(self, canvas):
        if len(self.aliens_fleet) != 0:   
            if len(self.aliens_fired_bullets) < self.max_fired_bullets: 
                choix = rd.randint(0,len(self.aliens_fleet)-1) 
                x,y = canvas.coords(self.aliens_fleet[choix].id) 
                self.aliens_fire(canvas,x,y) 
    
    
    def aliens_fire(self, canvas, x, y):
        if float(time.time()) - self.previous_shoot_time >= self.temps_ecart: 
            bullet = Bullet("alien")
            bullet = bullet.install_aliens_bullets(canvas,x,y) 
            self.aliens_fired_bullets.append(bullet) 
            self.previous_shoot_time = float(time.time()) 
    
   
    def move_aliens_bullets(self, canvas):
        for i in range(0,len(self.aliens_fired_bullets)):
            x1,y1,x2,y2 = canvas.bbox(self.aliens_fired_bullets[i].id) 
            if y1>int(canvas.cget("height")): 
                canvas.delete(self.aliens_fired_bullets[i].id)  
                del self.aliens_fired_bullets[i]  
                break;
            else:
                self.aliens_fired_bullets[i].move_in(canvas) 
    
    
    def animation_aliens_img(self, canvas):
        temps = float(time.time()) 
        if temps - self.animation_start_time > 1: 
            self.animation_start_time = temps 
            for i in range(len(self.aliens_fleet)):
                self.aliens_fleet[i].refresh_img(canvas) 
    
    
    def effet_boom(self,canvas,projectile):
        self.boom = PhotoImage(file="explosion.gif")
        x1,y1,x2,y2 = canvas.bbox(projectile.id) 
        x,y = x1+(x2-x1)/2, y1+(y2-y1)/2
        boom = canvas.create_image(x, y, image=self.boom, tags="boom")
        canvas.after(45,canvas.delete,boom) 
        
#------------- Alien ---------------#
class Alien(object):
    def __init__(self):
        self.id = None
        self.alive = True
        self.alien = PhotoImage(file="alien2.png") 
        self.alien_width = self.alien.width()
        self.alien_height = self.alien.height()
        
    def get_width(self):
        return self.alien_width
    def get_height(self):
        return self.alien_height
    
   
    def install_in(self, canvas, x, y): 
        
        self.id = canvas.create_image(x, y, image=self.alien, tags="alien")
        
        return self
    
    
    def move_in(self, canvas, dx, dy):
        canvas.move(self.id, dx, dy)
    
    
    def touched_by(self,canvas,projectile):
        canvas.delete(projectile.id)  
        canvas.delete(self.id)    
        self.alive = False  
        
    
    def refresh_img(self, canvas):
        if (self.alien.cget("file") == "alien2.png"): 
            self.alien = PhotoImage(file="rk1.png") 
            canvas.itemconfigure(self.id, image=self.alien) 
            self.alien = PhotoImage(file="alien2.png") 
            canvas.itemconfigure(self.id, image=self.alien) 
            
            
#------------- Defender ---------------#
class Defender(object):
    def __init__(self):
        
        self.defender_img = PhotoImage(file="rk1.png")
        self.life_img = PhotoImage(file="heartF.gif")
        self.boom = PhotoImage(file="explosion1.gif")
        self.width = self.defender_img.width()
        self.height = self.defender_img.height()
       
        self.canvas_height = Fleet().get_height()
        self.canvas_width = Fleet().get_width()
        
        self.id = None
        self.lifes = 3
        self.xi = self.canvas_width//2 - self.width
        self.yi = self.canvas_height - self.height
        self.move_delta = self.width/2 
        
        self.max_fired_bullets = 8
        self.fired_bullets = []
    
    
    def get_height(self):
        return self.height
    def get_width(self):
        return self.width
    
   
    def install_in(self, canvas):
       
        self.id = canvas.create_image(self.xi ,self.yi, image = self.defender_img , tags="defender")
    
    
    def display_defender_lifes(self, canvas):
        width = self.life_img.width() 
        height = self.life_img.height()
        inner_gap = width/2
        
        x = int(canvas.cget("width"))/2 - (width)*3/2 - inner_gap
        y = height * 3/4
        for i in range(self.lifes): #i < 3
            
            life = canvas.create_image(x, y, image = self.life_img ,tags="lifes")
            x += width + inner_gap
            
    
    def move_in(self , canvas , dx):   
        canvas.move(self.id, dx, 0)
    
   
    def fire(self, canvas):
        if len(self.fired_bullets) < self.max_fired_bullets: 
            bullet = Bullet("defender")
            bullet = bullet.install_in(canvas)   
            self.fired_bullets.append(bullet)    
    
   
    def move_bullet(self,canvas):
        for i in range(0,len(self.fired_bullets)):
            x1,y1,x2,y2 = canvas.bbox(self.fired_bullets[i].id)
            if y1<0: 
                canvas.delete(self.fired_bullets[i].id) 
                del self.fired_bullets[i]
                break
            else:
                self.fired_bullets[i].move_in(canvas)   
    
   
    def manage_defender_touched_by(self, canvas, fleet):
        x1, y1, x2, y2 = canvas.bbox(self.id)
        overlapped = canvas.find_overlapping(x1, y1, x2, y2)
        if len(overlapped)>1: 
            sortir = False
            for i in range(len(overlapped)):
                for j in range(len(fleet.aliens_fired_bullets)):
                    if overlapped[i] == fleet.aliens_fired_bullets[j].id: 
                        self.lifes -= 1 
                        self.touched_by(canvas) 
                        canvas.delete(fleet.aliens_fired_bullets[j].id) 
                        del fleet.aliens_fired_bullets[j]  
                        all_lifes = canvas.find_withtag("lifes") 
                        canvas.delete(all_lifes[-1]) 
                        sortir = True
                        break;
                if sortir == True: 
                    break;
    
   
   
    
   
    def touched_by(self,canvas):
        x,y = canvas.coords(self.id) 
        self.boom_effect(canvas,x,y)    
    
    
    def boom_effect(self, canvas, x, y):
        boom = canvas.create_image(x, y, image=self.boom, tags="boom")
        canvas.after(45,canvas.delete,boom) 
                    
#------------- Bullet ---------------#
class Bullet(object):
    def __init__(self, shooter):
        self.shooter = shooter
        self.id = None
        if self.shooter == "defender": 
            self.radius = 10
            self.color = "red"
            self.speed = 8
        elif self.shooter == "alien": 
            self.radius = 8
            self.color = "blue"
            self.speed = 3
    
    
    def install_in(self, canvas):
        if self.shooter=="defender":
            w,h = Defender().get_width() , Defender().get_height() 
            x,y = canvas.coords("defender")  
            x,y = x - w/4 , y - h 
            r = self.radius
            self.id = canvas.create_oval(x, y, x+r, y+r, fill = self.color)
            return self
    
    
    def install_aliens_bullets(self, canvas, x, y):
        if self.shooter=="alien":
            r = self.radius
            self.id = canvas.create_oval(x,y,x+r, y+r,fill=self.color)
            canvas.tag_lower(self.id)
            return self
    
    
    def move_in(self,canvas):
        if self.shooter=="defender":
            canvas.move(self.id, 0, -self.speed) 
        elif self.shooter=="alien":
            canvas.move(self.id, 0, self.speed)
             



            
#------------- Score ---------------#

class Score(object):
    def __init__(self):
        self.name = None
        self.points = 0
        self.winning = False 
        self.delai = None 
        self.start_time = int(time.time())
        self.delai_recommended = 2*60   
    
   
    def get_points(self):
        return self.points
    def set_name(self,Nname):
        self.name = Nname
    
    
    def refresh_score(self, end_time):
        temps = int(end_time)
        if self.delai_recommended - (temps - self.start_time) > 0: 
            self.points += self.delai_recommended - (temps - self.start_time)
        else: 
            self.points += 10
    
   
    def toFile(self,file):
        f = open(file,"w")
        json.dump(self.__dict__,f)
        f.close();
    
    
    @classmethod
    def fromFile(cls, file):
        try: 
            f = open(file,"r")
        except FileNotFoundError:  
            print("Fichier introuvable")
            snew=Score() 
        else:
            d = json.load(f)  
            snew = Score()
            snew.name = d["name"] 
            snew.points = d["points"]
            snew.delai = d["delai"]
            snew.winning = d["winning"]
            f.close();
        finally:
            return snew  
    
    
    def __str__(self):
        return str(self.name) +" -> "+ str(self.points)

        
#------------- Les Resultats ---------------#      
class Resultats(object):
    def __init__(self):
        self.lesResultats=[]
    
    
    def get_resultats(self):
        return self.lesResultats
    def ajout(self,score):
        self.lesResultats.append(score)
    
    
    def __str__(self):
        try:
            ch=str(self.lesResultats[0])
            for elt in self.lesResultats[1:]:
                ch=ch +" , "+ str(elt)
            return ch
        except IndexError: 
            return "Pas de score enregistrÃ©"
    
    
    def toFile(self,file):
        f = open(file,"w")
        tmp=[]
        for s in self.lesResultats:
            dic = {}
            dic["name"] = s.name
            dic["points"] = s.points
            dic["delai"] = s.delai
            dic["winning"] = s.winning
            tmp.append(dic)
        json.dump(tmp,f)
        f.close();
    
   
    @classmethod
    def fromFile(cls,file):
        try:
            f = open(file, "r")
        except FileNotFoundError: 
            print("Ce fichier n'existe pas")
            res=Resultats()
            res.lesResultats=[]
        else:
            tmp = json.load(f)
            liste = []
            for d in tmp:
                sc=Score()
                sc.name = d["name"]
                sc.points = d["points"]
                sc.delai = d["delai"]
                sc.winning = d["winning"]
                liste.append(sc)
            res = Resultats()
            res.lesResultats = liste
            f.close()
        finally:
            return res


toto=SpaceInvaders()
toto.play()
 
